In [1]:
import sys
sys.path.append('../RecSysRep/')

import Basics.Load as ld

URM_all, ICM_genre_all, ICM_subgenre_all, ICM_channel_all, ICM_event_all = ld.getCOOs()
# URM_train, URM_val = ld.getSplit(URM_train_val, 5678, 0.8)

URM_all_dataframe, ICM_genre_all_dataframe, ICM_subgenre_all_dataframe, ICM_channel_all_dataframe, ICM_event_all_dataframe = ld.getDataframes()

import numpy as np
from scipy.sparse import *

In [3]:
ICM_event_all_dataframe

,ItemID,EpisodeID,Match
0,0,121781,1.0
1,1,150102,1.0
2,1,349614,1.0
3,2,9174,1.0
4,2,20833,1.0
...,...,...,...
358065,18056,169988,1.0
358066,18056,198728,1.0
358067,18057,237262,1.0
358068,18058,96320,1.0


In [4]:
ICM_event_all = ICM_event_all.tocsr()
episodesPerShow = np.ediff1d(ICM_event_all.indptr)

In [5]:
print(episodesPerShow)
# episodesPerShow.sort()
print(episodesPerShow)

[1 2 3 ... 2 1 2]
[1 2 3 ... 2 1 2]


In [6]:
uniqueEpisodes = set(episodesPerShow)
# print(uniqueEpisodes)

import collections
counter=collections.Counter(episodesPerShow)
episodeFrequency = list(counter.items())
# counter.most_common(20)
invEF = [(sub[1], sub[0]) for sub in episodeFrequency]

In [7]:
print(invEF)

[(3849, 1), (3603, 2), (2328, 3), (1465, 4), (41, 27), (15, 50), (406, 7), (17, 55), (597, 6), (34, 31), (138, 15), (92, 0), (414, 8), (8, 81), (202, 12), (79, 21), (892, 5), (322, 9), (189, 13), (43, 30), (56, 24), (1, 735), (94, 20), (8, 116), (250, 11), (13, 57), (39, 28), (27, 34), (21, 53), (124, 18), (1, 294), (93, 19), (66, 22), (1, 927), (3, 318), (1, 238), (19, 47), (5, 157), (4, 124), (36, 29), (64, 23), (31, 32), (26, 35), (177, 14), (12, 69), (289, 10), (12, 70), (24, 39), (12, 59), (66, 25), (1, 2699), (11, 74), (1, 503), (11, 79), (11, 51), (153, 16), (31, 38), (2, 91), (4, 207), (99, 17), (52, 26), (26, 41), (18, 43), (3, 137), (15, 63), (13, 119), (2, 272), (8, 84), (19, 62), (4, 176), (16, 54), (4, 162), (6, 83), (2, 133), (1, 660), (16, 56), (2, 202), (35, 33), (10, 58), (16, 68), (6, 128), (2, 233), (15, 61), (2, 170), (2, 477), (12, 76), (3, 195), (24, 37), (2, 267), (2, 397), (3, 249), (2, 385), (34, 36), (11, 73), (15, 48), (2, 253), (16, 87), (2, 180), (1, 659), 

In [8]:
reshapedEPR = episodesPerShow.reshape(-1,1)
print(reshapedEPR)
print(len(reshapedEPR))

[[1]
 [2]
 [3]
 ...
 [2]
 [1]
 [2]]
18059


In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3, random_state=0).fit(reshapedEPR)

#you can see the labels with:
print(kmeans.labels_)
print(len(kmeans.labels_))
print(kmeans.cluster_centers_)

In [36]:
import pandas as pd

a = pd.DataFrame(episodesPerShow, columns = ['NumEpisodes'])
a

,NumEpisodes
0,1
1,2
2,3
3,1
4,1
...,...
18054,1
18055,1
18056,2
18057,1


In [42]:
num_cluster = 5
div = pd.qcut(a.NumEpisodes, num_cluster, labels=list(range(0, num_cluster)))
div

0        0
1        1
2        2
3        0
4        0
        ..
18054    0
18055    0
18056    1
18057    0
18058    1
Name: NumEpisodes, Length: 18059, dtype: category
Categories (5, int64): [0 < 1 < 2 < 3 < 4]

In [43]:
a["cluster"] = div

In [44]:
for i in range(0, num_cluster):
    print(a.loc[a['cluster'] == i].describe())

       NumEpisodes
count  3941.000000
mean      0.976656
std       0.151014
min       0.000000
25%       1.000000
50%       1.000000
75%       1.000000
max       1.000000
       NumEpisodes
count       3603.0
mean           2.0
std            0.0
min            2.0
25%            2.0
50%            2.0
75%            2.0
max            2.0
       NumEpisodes
count  3793.000000
mean      3.386238
std       0.486950
min       3.000000
25%       3.000000
50%       3.000000
75%       4.000000
max       4.000000
       NumEpisodes
count  3170.000000
mean      7.171609
std       1.976083
min       5.000000
25%       5.000000
50%       7.000000
75%       9.000000
max      11.000000
       NumEpisodes
count  3552.000000
mean     87.679336
std     220.377718
min      12.000000
25%      17.000000
50%      30.000000
75%      79.000000
max    5778.000000


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans

import matplotlib as mpl
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['axes.spines.right'] = False



# determine which K-Means cluster each point belongs to
cluster_id = KMeans(3).fit_predict(reshapedEPR)

# determine densities by cluster assignment and plot
fig, ax = plt.subplots()
bins = np.linspace(episodesPerShow.min(), episodesPerShow.max(), 40)
for ii in np.unique(cluster_id):
    subset = episodesPerShow[cluster_id==ii]
    ax.hist(subset, bins=bins, alpha=0.5, label=f"Cluster {ii}")
ax.legend()
plt.show()

In [23]:
import scipy.sparse as sps

profile_length = np.ediff1d(sps.csr_matrix(URM_all).indptr)
sorted_users = np.argsort(profile_length)
n_users = URM_all.shape[0]


input_km = [[profile_length[i]] for i in range(0, n_users)]

In [24]:
input_km

[[168],
 [609],
 [140],
 [51],
 [607],
 [157],
 [63],
 [181],
 [220],
 [258],
 [328],
 [456],
 [196],
 [220],
 [282],
 [235],
 [236],
 [162],
 [393],
 [247],
 [536],
 [616],
 [242],
 [194],
 [561],
 [694],
 [730],
 [525],
 [292],
 [338],
 [201],
 [816],
 [93],
 [322],
 [201],
 [1048],
 [281],
 [195],
 [431],
 [211],
 [188],
 [128],
 [668],
 [238],
 [436],
 [769],
 [261],
 [116],
 [313],
 [230],
 [185],
 [178],
 [306],
 [434],
 [36],
 [21],
 [482],
 [59],
 [69],
 [304],
 [95],
 [298],
 [504],
 [255],
 [288],
 [336],
 [207],
 [356],
 [525],
 [264],
 [705],
 [269],
 [768],
 [299],
 [399],
 [1027],
 [130],
 [702],
 [1061],
 [45],
 [295],
 [60],
 [173],
 [338],
 [353],
 [89],
 [1259],
 [143],
 [149],
 [594],
 [320],
 [139],
 [944],
 [349],
 [1176],
 [335],
 [454],
 [201],
 [68],
 [482],
 [245],
 [477],
 [233],
 [802],
 [147],
 [744],
 [655],
 [425],
 [551],
 [620],
 [321],
 [227],
 [585],
 [239],
 [450],
 [184],
 [537],
 [225],
 [258],
 [229],
 [779],
 [159],
 [236],
 [716],
 [159],
 [292],

In [25]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3, random_state=0).fit(input_km)

#you can see the labels with:
print(kmeans.labels_)
print(len(kmeans.labels_))
print(kmeans.cluster_centers_)

[1 2 1 ... 1 1 1]
13650
[[893.93963415]
 [192.60269154]
 [467.95357339]]


In [42]:
for i in range(0, n_users):
    print(kmeans.labels_[i], input_km[i])

1 [168]
2 [609]
1 [140]
1 [51]
2 [607]
1 [157]
1 [63]
1 [181]
1 [220]
1 [258]
1 [328]
2 [456]
1 [196]
1 [220]
1 [282]
1 [235]
1 [236]
1 [162]
2 [393]
1 [247]
2 [536]
2 [616]
1 [242]
1 [194]
2 [561]
0 [694]
0 [730]
2 [525]
1 [292]
2 [338]
1 [201]
0 [816]
1 [93]
1 [322]
1 [201]
0 [1048]
1 [281]
1 [195]
2 [431]
1 [211]
1 [188]
1 [128]
2 [668]
1 [238]
2 [436]
0 [769]
1 [261]
1 [116]
1 [313]
1 [230]
1 [185]
1 [178]
1 [306]
2 [434]
1 [36]
1 [21]
2 [482]
1 [59]
1 [69]
1 [304]
1 [95]
1 [298]
2 [504]
1 [255]
1 [288]
2 [336]
1 [207]
2 [356]
2 [525]
1 [264]
0 [705]
1 [269]
0 [768]
1 [299]
2 [399]
0 [1027]
1 [130]
0 [702]
0 [1061]
1 [45]
1 [295]
1 [60]
1 [173]
2 [338]
2 [353]
1 [89]
0 [1259]
1 [143]
1 [149]
2 [594]
1 [320]
1 [139]
0 [944]
2 [349]
0 [1176]
2 [335]
2 [454]
1 [201]
1 [68]
2 [482]
1 [245]
2 [477]
1 [233]
0 [802]
1 [147]
0 [744]
2 [655]
2 [425]
2 [551]
2 [620]
1 [321]
1 [227]
2 [585]
1 [239]
2 [450]
1 [184]
2 [537]
1 [225]
1 [258]
1 [229]
0 [779]
1 [159]
1 [236]
0 [716]
1 [159]
1 [292]

1 [51]
1 [307]
2 [428]
1 [257]
2 [622]
1 [192]
1 [110]
1 [116]
2 [337]
2 [333]
2 [603]
2 [649]
1 [320]
0 [733]
1 [218]
1 [113]
2 [383]
1 [299]
2 [654]
1 [118]
1 [134]
0 [1341]
1 [171]
1 [60]
2 [337]
1 [246]
2 [410]
1 [110]
1 [105]
0 [1000]
2 [588]
2 [346]
2 [468]
1 [284]
2 [412]
0 [1104]
1 [148]
1 [16]
1 [174]
1 [180]
1 [249]
1 [184]
1 [206]
2 [564]
2 [558]
1 [129]
1 [214]
2 [469]
2 [352]
2 [676]
2 [548]
1 [237]
1 [276]
1 [239]
0 [782]
0 [684]
2 [558]
2 [390]
0 [842]
1 [308]
2 [536]
1 [230]
2 [367]
1 [10]
2 [644]
1 [178]
2 [359]
0 [870]
1 [189]
1 [120]
1 [247]
1 [327]
1 [184]
2 [405]
2 [364]
1 [279]
1 [171]
1 [211]
2 [428]
1 [220]
0 [1619]
0 [1003]
2 [441]
0 [712]
1 [203]
2 [362]
1 [161]
1 [189]
1 [221]
1 [171]
1 [277]
1 [110]
0 [705]
1 [144]
2 [352]
1 [246]
1 [285]
1 [190]
2 [417]
0 [915]
1 [167]
0 [938]
0 [946]
2 [408]
2 [397]
0 [753]
1 [241]
0 [697]
1 [126]
0 [1058]
2 [388]
1 [243]
1 [252]
2 [438]
1 [202]
2 [348]
2 [636]
0 [887]
0 [723]
2 [633]
1 [272]
2 [362]
1 [135]
0 [820]
2 [445

2 [334]
2 [442]
2 [558]
1 [37]
1 [128]
1 [47]
2 [499]
2 [332]
2 [601]
1 [131]
1 [104]
1 [232]
2 [392]
2 [547]
1 [265]
1 [193]
1 [201]
2 [377]
2 [350]
1 [258]
1 [162]
0 [1181]
1 [34]
2 [639]
2 [379]
1 [199]
1 [133]
2 [551]
0 [744]
0 [813]
1 [59]
0 [770]
1 [144]
2 [408]
1 [322]
1 [202]
1 [307]
1 [128]
1 [275]
1 [320]
1 [181]
2 [379]
2 [668]
2 [512]
2 [419]
2 [334]
0 [890]
1 [150]
0 [774]
1 [236]
1 [268]
1 [139]
1 [255]
0 [748]
1 [251]
2 [547]
2 [549]
1 [185]
1 [63]
1 [275]
2 [619]
1 [39]
1 [206]
2 [457]
2 [407]
0 [808]
2 [432]
1 [217]
2 [612]
1 [320]
1 [139]
1 [309]
1 [201]
1 [218]
1 [142]
2 [473]
1 [95]
1 [107]
1 [314]
2 [417]
2 [642]
1 [99]
2 [450]
2 [489]
0 [754]
2 [554]
2 [537]
1 [37]
2 [437]
1 [150]
0 [1109]
1 [288]
2 [640]
1 [325]
2 [568]
2 [583]
0 [847]
1 [108]
2 [402]
0 [734]
2 [585]
2 [384]
1 [162]
2 [457]
2 [356]
0 [968]
2 [524]
1 [329]
2 [518]
2 [434]
2 [439]
2 [529]
0 [802]
1 [218]
1 [329]
2 [466]
1 [170]
2 [434]
1 [80]
2 [450]
1 [322]
1 [230]
2 [373]
1 [98]
2 [479]
1 [116]
1

2 [511]
1 [291]
1 [14]
0 [926]
1 [296]
1 [304]
2 [550]
0 [826]
2 [405]
1 [144]
2 [399]
2 [367]
2 [489]
1 [79]
2 [524]
0 [721]
1 [82]
2 [668]
2 [379]
0 [1295]
1 [289]
1 [48]
2 [382]
0 [690]
2 [439]
2 [454]
2 [567]
2 [556]
2 [341]
2 [555]
2 [500]
2 [397]
2 [639]
2 [633]
2 [588]
2 [517]
1 [71]
1 [199]
2 [472]
2 [591]
1 [147]
2 [369]
1 [196]
0 [712]
1 [293]
2 [456]
2 [468]
1 [66]
1 [282]
0 [1132]
2 [674]
2 [656]
1 [65]
2 [346]
1 [65]
1 [181]
1 [100]
1 [60]
1 [65]
2 [477]
1 [300]
2 [660]
2 [505]
2 [596]
2 [358]
1 [78]
1 [220]
1 [308]
1 [293]
2 [667]
2 [344]
0 [899]
1 [67]
2 [420]
1 [259]
1 [116]
2 [379]
1 [140]
2 [621]
2 [425]
1 [102]
1 [107]
2 [441]
0 [829]
1 [233]
2 [386]
2 [556]
2 [416]
1 [184]
0 [1163]
2 [462]
1 [243]
0 [716]
1 [172]
1 [105]
1 [285]
1 [87]
1 [176]
1 [65]
1 [20]
1 [150]
1 [200]
2 [343]
2 [527]
2 [349]
1 [280]
1 [217]
0 [774]
2 [620]
2 [550]
1 [146]
2 [632]
2 [421]
2 [505]
1 [73]
0 [1802]
2 [592]
2 [366]
1 [238]
2 [552]
2 [582]
2 [367]
2 [620]
1 [232]
1 [232]
1 [234]
2 [4

2 [537]
1 [320]
1 [109]
2 [411]
1 [312]
1 [167]
1 [247]
1 [320]
1 [103]
0 [905]
1 [169]
1 [304]
1 [142]
2 [490]
2 [593]
2 [486]
1 [281]
1 [83]
1 [201]
1 [266]
2 [406]
1 [53]
2 [526]
1 [164]
1 [55]
2 [607]
0 [914]
1 [312]
0 [760]
2 [407]
1 [218]
2 [473]
0 [748]
0 [826]
2 [419]
1 [295]
1 [183]
2 [467]
1 [235]
2 [355]
1 [296]
2 [481]
0 [811]
2 [421]
2 [528]
2 [457]
1 [184]
1 [290]
1 [276]
1 [163]
0 [1231]
0 [775]
1 [147]
2 [411]
0 [831]
2 [477]
0 [844]
1 [78]
1 [41]
1 [114]
1 [319]
1 [135]
1 [281]
2 [434]
2 [341]
1 [15]
2 [333]
1 [159]
2 [394]
1 [122]
1 [14]
1 [296]
0 [1026]
0 [823]
0 [784]
2 [443]
2 [468]
2 [405]
0 [710]
1 [308]
1 [128]
1 [128]
1 [23]
0 [717]
2 [538]
2 [476]
1 [244]
2 [344]
1 [165]
2 [393]
1 [226]
1 [156]
2 [496]
2 [563]
1 [210]
1 [264]
1 [257]
1 [141]
1 [100]
1 [316]
1 [41]
1 [116]
2 [343]
1 [209]
2 [435]
1 [112]
2 [375]
2 [524]
2 [345]
1 [38]
1 [297]
1 [156]
1 [177]
1 [112]
1 [219]
0 [754]
1 [177]
2 [655]
1 [16]
2 [654]
2 [583]
2 [512]
0 [917]
0 [1004]
2 [468]
0 [722]


0 [900]
1 [20]
2 [338]
1 [284]
1 [53]
2 [502]
2 [547]
1 [23]
2 [427]
2 [597]
2 [448]
1 [218]
1 [184]
0 [890]
1 [212]
1 [211]
2 [545]
1 [266]
1 [161]
1 [61]
2 [649]
1 [133]
1 [329]
1 [105]
1 [31]
2 [378]
1 [254]
2 [454]
1 [301]
1 [299]
2 [344]
1 [324]
1 [279]
1 [88]
2 [558]
0 [779]
1 [278]
1 [162]
2 [416]
1 [185]
1 [182]
2 [373]
0 [1318]
1 [116]
1 [224]
2 [562]
1 [222]
2 [642]
2 [488]
0 [1195]
2 [674]
0 [683]
2 [622]
0 [1016]
0 [837]
2 [539]
2 [479]
2 [543]
2 [339]
1 [235]
2 [558]
1 [241]
1 [267]
2 [402]
1 [184]
2 [391]
2 [416]
2 [339]
1 [236]
2 [538]
2 [547]
0 [901]
2 [364]
1 [283]
1 [245]
2 [573]
2 [465]
1 [117]
1 [277]
1 [142]
2 [415]
2 [413]
1 [203]
2 [531]
2 [486]
1 [86]
2 [416]
2 [392]
2 [429]
0 [846]
2 [360]
0 [690]
2 [425]
1 [177]
1 [44]
2 [581]
2 [400]
2 [345]
2 [394]
2 [430]
1 [227]
0 [1466]
1 [61]
1 [54]
2 [431]
1 [244]
2 [366]
1 [97]
1 [297]
1 [242]
1 [214]
1 [119]
1 [219]
2 [569]
1 [325]
0 [683]
2 [481]
1 [223]
1 [217]
0 [681]
1 [43]
0 [813]
2 [665]
1 [280]
2 [391]
1 [86]
1

2 [338]
2 [341]
1 [152]
2 [359]
2 [332]
1 [196]
1 [245]
1 [180]
1 [197]
0 [977]
1 [201]
0 [941]
1 [14]
1 [309]
2 [513]
1 [215]
0 [757]
2 [531]
1 [173]
1 [69]
1 [174]
2 [547]
1 [240]
1 [169]
2 [411]
1 [105]
2 [461]
2 [533]
1 [186]
1 [260]
0 [845]
0 [695]
1 [328]
2 [331]
0 [773]
2 [393]
1 [284]
0 [838]
2 [342]
0 [839]
1 [165]
1 [223]
2 [360]
2 [358]
1 [125]
2 [604]
1 [219]
0 [784]
2 [407]
1 [300]
1 [286]
2 [531]
1 [255]
1 [269]
0 [1059]
2 [373]
2 [431]
1 [322]
1 [128]
1 [81]
2 [405]
1 [254]
2 [422]
1 [280]
0 [740]
2 [615]
2 [477]
1 [49]
1 [313]
1 [195]
1 [234]
1 [161]
1 [314]
2 [524]
1 [271]
1 [88]
1 [266]
2 [379]
2 [451]
1 [289]
0 [1020]
2 [467]
2 [407]
1 [268]
1 [102]
0 [1138]
0 [705]
1 [298]
1 [149]
1 [9]
2 [575]
2 [479]
2 [360]
2 [543]
2 [343]
1 [324]
1 [314]
1 [292]
1 [314]
2 [447]
0 [855]
2 [634]
2 [390]
0 [1167]
1 [326]
1 [315]
2 [374]
1 [94]
2 [649]
0 [739]
1 [190]
1 [229]
1 [271]
2 [490]
1 [286]
1 [277]
1 [258]
2 [469]
1 [152]
2 [630]
2 [359]
2 [439]
1 [75]
1 [302]
2 [454]
1 [58

1 [139]
1 [214]
1 [71]
2 [420]
1 [199]
2 [420]
1 [268]
0 [991]
0 [1077]
1 [196]
2 [527]
2 [668]
2 [475]
2 [612]
2 [602]
0 [830]
1 [67]
0 [774]
2 [578]
1 [215]
1 [258]
1 [289]
2 [364]
1 [178]
1 [99]
1 [147]
1 [142]
2 [587]
0 [797]
1 [201]
1 [222]
2 [391]
1 [48]
2 [563]
2 [452]
2 [483]
2 [522]
1 [17]
2 [405]
1 [137]
2 [428]
2 [496]
0 [812]
1 [60]
2 [355]
2 [507]
1 [106]
1 [259]
1 [196]
2 [479]
2 [349]
0 [696]
2 [348]
1 [218]
0 [1205]
2 [639]
0 [703]
2 [411]
1 [178]
2 [436]
1 [293]
2 [375]
1 [323]
2 [626]
1 [199]
2 [482]
0 [701]
2 [437]
2 [413]
1 [187]
2 [411]
1 [235]
2 [359]
2 [352]
1 [112]
1 [286]
2 [380]
1 [196]
2 [534]
1 [110]
2 [403]
0 [812]
1 [154]
2 [555]
1 [282]
1 [217]
2 [476]
2 [338]
2 [552]
1 [184]
1 [308]
2 [520]
1 [129]
1 [134]
2 [450]
2 [376]
1 [306]
0 [1035]
2 [616]
2 [458]
2 [535]
0 [791]
0 [752]
1 [197]
2 [338]
1 [142]
1 [68]
0 [786]
2 [377]
2 [588]
1 [153]
1 [37]
2 [386]
1 [324]
1 [159]
1 [159]
1 [131]
1 [62]
1 [104]
2 [648]
1 [51]
1 [278]
0 [758]
1 [178]
1 [256]
1 [313]

1 [190]
1 [285]
1 [143]
2 [661]
2 [508]
2 [477]
2 [579]
0 [996]
2 [347]
2 [443]
0 [727]
1 [325]
2 [348]
1 [313]
2 [632]
2 [495]
0 [1024]
1 [156]
1 [68]
2 [386]
2 [470]
2 [357]
1 [214]
1 [286]
1 [144]
1 [269]
2 [615]
1 [175]
2 [480]
2 [390]
0 [709]
1 [57]
2 [473]
1 [249]
1 [201]
1 [278]
1 [328]
0 [695]
0 [892]
1 [102]
0 [923]
2 [660]
1 [37]
1 [61]
1 [179]
1 [100]
1 [88]
1 [63]
2 [346]
1 [189]
1 [218]
2 [520]
1 [296]
2 [500]
2 [676]
2 [503]
1 [216]
1 [230]
2 [406]
1 [65]
1 [189]
2 [557]
2 [389]
1 [313]
2 [355]
1 [228]
1 [280]
2 [551]
1 [95]
1 [246]
1 [298]
1 [314]
1 [235]
1 [117]
2 [496]
1 [140]
2 [342]
1 [181]
1 [122]
1 [186]
2 [418]
2 [574]
1 [178]
1 [275]
1 [293]
2 [489]
1 [48]
2 [547]
2 [525]
1 [139]
0 [694]
1 [26]
2 [632]
1 [42]
2 [518]
2 [596]
2 [427]
2 [606]
0 [752]
2 [552]
1 [85]
2 [538]
1 [261]
1 [324]
1 [105]
1 [320]
2 [552]
1 [43]
2 [399]
2 [419]
2 [638]
1 [173]
2 [470]
2 [365]
1 [256]
2 [412]
1 [252]
1 [142]
2 [553]
1 [39]
1 [275]
2 [642]
1 [280]
0 [730]
1 [323]
2 [383]
1 [18

2 [554]
2 [356]
2 [524]
2 [370]
1 [251]
1 [306]
2 [340]
2 [617]
2 [518]
1 [122]
1 [208]
2 [376]
0 [814]
2 [547]
1 [129]
1 [102]
2 [441]
2 [377]
2 [363]
2 [394]
1 [167]
2 [369]
2 [646]
1 [214]
2 [575]
1 [132]
2 [566]
1 [133]
2 [343]
1 [50]
1 [118]
1 [250]
2 [431]
2 [400]
1 [264]
1 [217]
1 [285]
2 [646]
1 [58]
2 [431]
1 [285]
1 [120]
2 [558]
1 [281]
1 [169]
2 [545]
0 [946]
1 [300]
2 [517]
1 [275]
2 [338]
2 [441]
1 [327]
1 [314]
1 [201]
1 [306]
0 [1333]
2 [570]
1 [90]
1 [144]
0 [740]
1 [325]
1 [325]
2 [613]
0 [844]
2 [564]
2 [546]
1 [125]
0 [910]
1 [318]
1 [196]
1 [239]
1 [145]
2 [405]
2 [428]
1 [207]
1 [131]
1 [33]
2 [335]
1 [94]
2 [357]
2 [438]
2 [588]
1 [269]
1 [88]
2 [404]
1 [265]
1 [267]
1 [132]
1 [131]
1 [317]
1 [164]
1 [44]
2 [674]
1 [76]
1 [44]
2 [345]
2 [429]
1 [293]
2 [630]
2 [525]
1 [202]
0 [745]
2 [541]
1 [116]
2 [402]
1 [291]
1 [302]
2 [360]
1 [45]
1 [309]
1 [38]
1 [263]
1 [46]
0 [1024]
1 [101]
2 [420]
2 [357]
0 [999]
2 [391]
1 [314]
1 [140]
2 [586]
1 [297]
1 [109]
1 [264]
1 

In [47]:
group_2 = np.where(kmeans.labels_ == 0)[0]
group_1 = np.where(kmeans.labels_ == 1)[0]
group_0 = np.where(kmeans.labels_ == 2)[0]

In [52]:
groups = np.array([group_0, group_1, group_2], dtype=object)

In [53]:
np.save("../data/groups", groups)

In [54]:
groups[0]

array([    1,     4,    11, ..., 13639, 13644, 13645], dtype=int64)

In [58]:
grps = np.arange(0, n_users)
grp = np.delete(grps, groups[0])

In [59]:
print(grp)
print(groups[0])

[    0     2     3 ... 13647 13648 13649]
[    1     4    11 ... 13639 13644 13645]


In [2]:
grp = ld.load_group()

In [3]:
grp

array([array([    1,     4,    11, ..., 13639, 13644, 13645], dtype=int64),
       array([    0,     2,     3, ..., 13647, 13648, 13649], dtype=int64),
       array([   25,    26,    31, ..., 13632, 13633, 13636], dtype=int64)],
      dtype=object)

In [9]:
a = [1,2,3]
b = [1]
c = [2]

d = [a,b,c]

e = np.array(d)

C:\Users\franc\AppData\Local\Temp/ipykernel_7044/2872807733.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  e = np.array(d)


In [10]:
e

array([list([1, 2, 3]), list([1]), list([2])], dtype=object)

In [11]:
for i in range(3, -1, -1):
    print(i)

3
2
1
0
